## Setup

In [1]:
from siuba.sql import LazyTbl
from siuba import *
from sqlalchemy import create_engine
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact

from IPython.display import display, FileLink

In [2]:
engine = create_engine("bigquery://cal-itp-data-infra")

## Create tables

In [4]:
tbl_status = LazyTbl(engine, "gtfs_schedule.calitp_status")
tbl_validation_notices = LazyTbl(engine, 'test_views.validation_notices')
sheet = pd.read_csv("https://docs.google.com/spreadsheets/d/105oar4q_Z3yihDeUlP-VnYpJ0N9Mfs7-q4TnribqYLU/gviz/tq?tqx=out:csv&gid=471807468")

## Fetch district specific validator results

In [5]:
options = sheet["Caltrans District"].dropna().sort_values().astype(int).unique()

In [6]:
def get_district_validations(district):
    district = sheet[lambda d: d["Caltrans District"] == district]
    itp_ids = district.ITP_ID.astype(int).tolist()

    itp_id_str = ", ".join(map(str, itp_ids))

    tbl_notices_subset = (
        tbl_validation_notices
        >> filter(_.calitp_itp_id.isin(itp_ids))
        >> left_join(
            _,
            tbl_status >> select(_.itp_id, _.url_number, _.agency_name),
            {"calitp_itp_id": "itp_id", "calitp_url_number": "url_number"},
        )
        # move useful columns to the left
        >> select(_.calitp_itp_id, _.calitp_url_number, _.agency_name, _.contains(""))
    )
    
    return tbl_notices_subset >> collect()


In [8]:
@interact(district=options)
def ui_download_validations(district):
    print("getting results...")
    df = get_district_validations(district)
    
    fname = "./district_%s_validations.csv" % int(district)
    df.to_csv(fname)
    return FileLink(fname, result_html_prefix="Download file: ")

interactive(children=(Dropdown(description='district', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=…